# Comparing the competing impacts of binary physics and galactic potentials

## The goal and plan

In this case study, the question we want to answer is **"what is the relative importance of binary physics and the galactic potential in determining the spatial distributions of massive stars"**. In order to do this, let's compare the final galactocentric $x$ and $z$ positions of massive stars at present day (i.e. stars with more than $7 {\rm M_\odot}$ and a stellar type less than $10$), varying the mean magnitude of supernova natal kicks and the galactic potential used.

In [157]:
import cogsworth
import numpy as np
import astropy.units as u
import gala.potential as gp
from gala.units import galactic
import seaborn as sns
import matplotlib.pyplot as plt
import warnings

In [2]:
%config InlineBackend.figure_format = 'retina'

plt.rc('font', family='serif')
plt.rcParams['text.usetex'] = False
fs = 20

# update various fontsizes to match
params = {'figure.figsize': (12, 8),
          'legend.fontsize': 0.7 * fs,
          'axes.labelsize': fs,
          'xtick.labelsize': 0.7 * fs,
          'ytick.labelsize': 0.7 * fs,
          'axes.linewidth': 1.1,
          'xtick.major.size': 7,
          'xtick.minor.size': 4,
          'ytick.major.size': 7,
          'ytick.minor.size': 4}
plt.rcParams.update(params)

## Simulate the populations
To start, let's simulate the population. Specifically, let's simulate a population of stars formed in the last 100 million years in the Milky Way with a 100% binary fraction.

In [3]:
# only want the last 100 Myr of star formation
max_ev_time = 100 * u.Myr

### Define some star formation histories and potentials

First, let's define two different SFH and potential models that approximately represent a quasi-isothermal MW thin disc and the Carina dwarf galaxy, but using a fixed metallicity for both.

In [24]:
# use a quasi-isothermal disk model for the star formation history, but with a more recent history
class RecentDisc(cogsworth.sfh.QuasiIsothermalDisk):
    def draw_lookback_times(self, size=None, component="low_alpha_disc"):
        U = np.random.rand(size)
        norm = 1 / (self.tsfr * np.exp(-self.galaxy_age / self.tsfr) * (np.exp(max_ev_time / self.tsfr) - 1))
        tau = self.tsfr * np.log((U * np.exp(self.galaxy_age / self.tsfr)) / (norm * self.tsfr) + 1)

        return tau
    
    def get_metallicity(self):
        return np.repeat(0.02, len(self)) * u.dimensionless_unscaled

# simply the Milky Way potential
pot = gp.MilkyWayPotential2022()

In [34]:
# approximate the Carina dwarf galaxy recent star formation history
class Carina(cogsworth.sfh.SpheroidalDwarf):
    def __init__(self, size, mass=8.67e8 * u.Msun, J_0_star=0.677 * u.kpc*u.km/u.s,
                       alpha=0.946, eta=0.5, **kwargs):
        super().__init__(size=size, mass=mass, J_0_star=J_0_star, alpha=alpha, eta=eta, **kwargs)

    def draw_lookback_times(self, size=None, component="low_alpha_disc"):
        U = np.random.rand(size)
        norm = 1 / (self.tsfr * np.exp(-self.galaxy_age / self.tsfr) * (np.exp(max_ev_time / self.tsfr) - 1))
        tau = self.tsfr * np.log((U * np.exp(self.galaxy_age / self.tsfr)) / (norm * self.tsfr) + 1)

        return tau
    
    def get_metallicity(self):
        return np.repeat(0.02, len(self)) * u.dimensionless_unscaled
    
# use a simple NFW potential for the Carina dwarf galaxy
carina_pot = gp.NFWPotential(m=8.67e8 * u.Msun, r_s=1.0, units=galactic)

### Sample initial conditions

For the most consistent comparison, we'll use the same initial conditions for each population. Let's start by sampling these for the base population.

In [139]:
base_pop = cogsworth.pop.Population(
    2_000_000,
    processes=6,
    m1_cutoff=7,
    sfh_model=RecentDisc,
    galactic_potential=pot,
    max_ev_time=max_ev_time,
    BSE_settings={"binfrac": 1.0},
    store_entire_orbits=False
)
base_pop.sample_initial_binaries()

### Evolve binary physics variations

Now let's evolve this for two choices of the mean supernova natal kick magnitude:

- the default choice of $265 \, {\rm km /s}$
- a variation with extremely strong kicks with a mean of $1000 \, {\rm km /s}$

In [140]:
# copy the base population
disc_low_sigma = base_pop[:]
disc_high_sigma = base_pop[:]

# adjust sigma for both types of kicks
disc_high_sigma.BSE_settings["sigma"] = 1000
disc_high_sigma.BSE_settings["sigmadiv"] = -1000

# evolve both
disc_low_sigma.perform_stellar_evolution()
disc_high_sigma.perform_stellar_evolution()

### Mask the populations

Mask the population to only retain systems that experienced a supernova and are now either in a bound binary with a star or are a star from a disrupted binary.

In [141]:
KSTAR_MAX = 10

def mask_population(p):
    # get the binarys that have at least one SN
    has_sn_bin_nums = p.bpp[p.bpp["evol_type"].isin([15, 16])]["bin_num"].unique()
    
    # reduce population
    p_sn = p[has_sn_bin_nums]
    
    # get bound/merged binaries that have a star in them
    stars_in_binaries = (p_sn.final_bpp["sep"] >= 0.0) & ((p_sn.final_bpp["kstar_1"] < KSTAR_MAX)
                                                       |  (p_sn.final_bpp["kstar_2"] < KSTAR_MAX))
    
    # get disrupted binaries where at least one component is still a star
    disrupted_stars = (p_sn.final_bpp["sep"] == -1) & ((p_sn.final_bpp["kstar_1"] < KSTAR_MAX)
                                                     | (p_sn.final_bpp["kstar_2"] < KSTAR_MAX))
    return p_sn[p_sn.bin_nums[stars_in_binaries | disrupted_stars]]

In [142]:
disc_low_sigma = mask_population(disc_low_sigma)

In [143]:
disc_high_sigma = mask_population(disc_high_sigma)

### Integrate through different potentials
Now, for each value of sigma we can evolve it through both the disc and dwarf galaxy.

In [144]:
# copy the other populations
carina_low_sigma = disc_low_sigma[:]
carina_high_sigma = disc_high_sigma[:]

# change the SFH and potential
carina_low_sigma.sfh_model = Carina
carina_low_sigma.galactic_potential = carina_pot
carina_high_sigma.sfh_model = Carina
carina_high_sigma.galactic_potential = carina_pot

# resample the star formation history
carina_low_sigma.sample_initial_galaxy()
carina_high_sigma.sample_initial_galaxy()

In [159]:
# evolve them all through their respective galaxies
with warnings.catch_warnings():
    warnings.filterwarnings("ignore", message=".*bad orbit.*")
    disc_low_sigma.perform_galactic_evolution()
    disc_high_sigma.perform_galactic_evolution()
    carina_low_sigma.perform_galactic_evolution()
    carina_high_sigma.perform_galactic_evolution()

6178it [00:05, 1194.58it/s]                          
6596it [00:05, 1139.80it/s]                          
4918it [00:03, 1235.13it/s]                          
5262it [00:04, 1152.53it/s]                          


## Get present day positions of stars

Now we need some functions to split the populations into the correct subpopulations and also a way to access the final positions of only stars. The rest of the code in this case study follows very similarly to the previous one.

In [146]:
def get_x_z(p):
    """Get the final x-z positions of the stars in the population"""
    # bound systems need at least one star
    bound_pos = p.final_pos[:len(p)][~p.disrupted & ((p.final_bpp["kstar_1"] <= KSTAR_MAX)
                                                   | (p.final_bpp["kstar_2"] <= KSTAR_MAX))]
    # disrupted primary (secondary) systems need the primary (secondary) to be a star
    dis_pos_1 = p.final_pos[:len(p)][p.disrupted & (p.final_bpp["kstar_1"] <= KSTAR_MAX)]
    dis_pos_2 = p.final_pos[len(p):][p.final_bpp["kstar_2"][p.disrupted] <= KSTAR_MAX]

    # combine them all and return x, z
    all_pos = np.concatenate([bound_pos, dis_pos_1, dis_pos_2])
    return all_pos[:, 0], all_pos[:, 2]

## Visualise the results

We'll visualise this with a single density plot using a Kernel Density Estimator (KDE). Scipy provides a
simple gaussian one and we can build around it to mirror the density on the plot.

### Calculate densities for each population

This uses the ``scipy`` ``gaussian_kde`` function to calculate the overall $x$-$z$ density and then reflects it into a specific half/quadrant depending on user input

In [136]:
from scipy.stats import gaussian_kde

def get_mirrored_density(x, y, x_range, y_range, mirror_x=None, mirror_y=None):
    """Get the mirrored density of a 2D distribution"""
    X, Y = np.meshgrid(x_range, y_range)
    densities = gaussian_kde([x, y]).evaluate([X.flatten(), Y.flatten()]).reshape((len(x_range), len(y_range)))
    
    # mirror left or right if requested
    if mirror_x == "left":
        densities = np.where(X <= 0, densities + np.fliplr(densities), 0)
    elif mirror_x == "right":
        densities = np.where(X >= 0, densities + np.fliplr(densities), 0)
    
    # mirror up or down if requested
    if mirror_y == "up":
        densities = np.where(Y >= 0, densities + np.flipud(densities), 0)
    elif mirror_y == "down":
        densities = np.where(Y <= 0, densities + np.flipud(densities), 0)
        
    return X, Y, densities

We can apply the function to each of the subpopulations with different inputs using the following loop

In [147]:
# define the range we want to plot
x_range = np.linspace(-30, 30, 201) * u.kpc
z_range = np.linspace(-15, 15, 201) * u.kpc

# set up a dictionary to store the data
data = {
    "disc_low_sigma": {},
    "disc_high_sigma": {},
    "carina_low_sigma": {},
    "carina_high_sigma": {}
}

# loop over the populations and get the density (mirrored in the correct way)
for pop, label, mirror_x, mirror_y in zip([disc_low_sigma, disc_high_sigma, carina_low_sigma, carina_high_sigma],
                                          ["disc_low_sigma", "disc_high_sigma", "carina_low_sigma", "carina_high_sigma"],
                                          ["left", "right", "left", "right"],
                                          ["up", "up", "down", "down"]):
    # get the x and z positions
    x, z = get_x_z(pop)
    # only keep the stars within the range we want to plot
    x, z = x[(x >= x_range.min()) & (x <= x_range.max()) & (z >= z_range.min()) & (z <= z_range.max())],\
        z[(z >= z_range.min()) & (z <= z_range.max()) & (x >= x_range.min()) & (x <= x_range.max())]
    
    # get the mirrored density
    X, Z, data[label]["density"] = get_mirrored_density(x, z,
                                                        x_range, z_range, mirror_x, mirror_y)
    
    # store the x and z positions as well
    data[label]["x"], data[label]["z"] = x, z

### Plot the densities

Now let's make a nice plot showing the density distributions! This plot shows the 2D density distribution up to the 98th percentile, with the remaining outliers plotted with scatter points. There's also a marginal distribution showing the KDE in $z$ for each of the populations.

In [109]:
def _iso_level_to_levels(data, iso_levels):
    # convert a list of isoprobability levels to actual levels for contourf plots
    # (don't worry too much about the details here ;) )
    isoprop = np.asarray(iso_levels)
    values = np.ravel(data)
    sorted_values = np.sort(values)[::-1]
    normalized_values = np.cumsum(sorted_values) / values.sum()
    idx = np.searchsorted(normalized_values, 1 - isoprop)
    levels = np.take(sorted_values, idx, mode="clip")
    return np.unique(levels)

## Discussion and explanations

The resulting density distribution show that both binary physics and the galactic potential can have a large impact on where we find massive stars (in particular those that experienced a supernova in this case). Increasing the strength of supernova natal kicks in the Milky Way disc case significantly increases the scale height, but a more drastic effect is seen in the weaker (and more spherical) Carina dwarf galaxy potential, which allows stars to spread further.